In [1]:
import torch
torch.__version__

'1.7.0+cu101'

In [2]:
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader,TensorDataset,random_split
from torchvision.transforms import transforms

In [3]:
class SFTT(nn.Module):

  def __init__(self):
    super().__init__()
    self.network = nn.Sequential(
        nn.Conv2d(1,64,kernel_size=3,stride=1,padding=1),
        nn.BatchNorm2d(64),
        nn.ReLU(),
        nn.MaxPool2d(2,2),
        nn.Conv2d(64,128,kernel_size=3,stride=1,padding=1),
        nn.BatchNorm2d(128),
        nn.ReLU(),
        nn.MaxPool2d(2,2),

        nn.Conv2d(128,128,kernel_size=3,stride=1,padding=1),
        nn.BatchNorm2d(128),
        nn.ReLU(),
        nn.MaxPool2d(2,2),
        nn.Conv2d(128,256,kernel_size=3,stride=1,padding=1),
        nn.BatchNorm2d(256),
        nn.ReLU(),
        nn.MaxPool2d(2,2),
        nn.AdaptiveAvgPool2d(1),

        nn.Flatten(),
        nn.Linear(256,128),
        nn.Tanh(),
        nn.Linear(128,64),
        nn.Tanh(),
        nn.Linear(64,1),
        nn.Sigmoid()
    )

  def forward(self,xb):
    out = self.network(xb)
    return out
  
  def training_step(self,batch):
    features,label = batch
    out = self(features)
    loss = F.binary_cross_entropy(out,label)
    return loss

  def validation_step(self,batch):
    features,label = batch
    out = self(features)
    loss = F.binary_cross_entropy(out,label)
    acc = accuracy(out,label)
    return {"val_loss": loss.detach(),"val_acc": acc}

  def validation_epoch_end(self,outputs):
    batch_loss = [x['val_loss'] for x in outputs]
    epoch_loss = torch.stack(batch_loss).mean()
    batch_acc = [x['val_acc'] for x in outputs]
    epoch_acc = torch.stack(batch_acc).mean()
    return {"val_loss":epoch_loss.item(),"val_acc":epoch_acc.item()}

  def epoch_end(self,num_epoch,results):
    print("num_epoch: {}, train_loss: {:.2f}, val_loss: {:.2f}, val_acc: {:.2f}".format(num_epoch+1,results['train_loss'],results['val_loss'], results['val_acc']))

In [4]:
def accuracy(out,label):
  out = (out>0.5)
  pred = (out == label).sum()
  return pred/out.shape[0]

def evaluate(model,val_loader):
  outputs = [model.validation_step(batch) for batch in val_loader]
  return model.validation_epoch_end(outputs)

def fit(num_epochs,lr,train_loader,val_loader,model,opt_func=torch.optim.Adam):
  optimizer = opt_func(model.parameters(),lr)
  history = []
  for epoch in range(num_epochs):
    train_losses = []
    for batch in train_loader:
      loss = model.training_step(batch)
      train_losses.append(loss)
      loss.backward()
      optimizer.step()
      optimizer.zero_grad()

    results = evaluate(model,val_loader)
    train_loss = torch.stack(train_losses).mean().item()
    results['train_loss'] = train_loss
    model.epoch_end(epoch,results)
    history.append(results)
  return history

In [5]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

In [6]:
device = get_default_device()
device

device(type='cpu')

In [7]:
model = SFTT()
model = model.to(device)

In [10]:
train_dl = DeviceDataLoader(train_dl, device)
val_dl = DeviceDataLoader(val_dl, device)

In [11]:
history = fit()